In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

## Load results from WeatherPy

In [2]:
# Loading in csv from WeatherPy activity into a DataFrame
weather_path = '../WeatherPy/Output/cities.csv'
weather_df = pd.read_csv(weather_path)

## Humidity Heatmap

In [7]:
# Configuring gmaps with the api key
gmaps.configure(api_key=g_key)

# Layout of heatmap figures
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [5]:
# Storing latitude and longitude in a DataFrame as floats
locations = weather_df[['Lat', 'Lng']].astype(float)
# Storing humidity for weight of heatmap
humidity = weather_df['Humidity'].astype(float)

In [8]:
# Creating heatmap of all locations 
fig = gmaps.figure(figure_layout)
# Heatmap layer weighted based on humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                max_intensity=100, point_radius=1)
# Applying heatmap layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create a new DataFrame fitting weather criteria
Ideal Weather Condition: Wind Speed < 7 mph, max temperature $\leq$ 75°F, humidity < 40, and cloudiness < 25

In [40]:
# Keeping only the rows with the ideal weather conditions
ideal_weather = weather_df.loc[(weather_df['Wind Speed'] < 7) &
                               (weather_df['Max Temp'] <= 75) &
                               (weather_df['Humidity'] < 40) &
                               (weather_df['Cloudiness'] < 25)]

# Dropping na rows
ideal_weather = ideal_weather.dropna()

ideal_weather.describe()

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,8.000000,8.000000,8.000000,8.00000,8.000000,8.000000,8.000000,8.000000e+00
mean,346.000000,27.738975,15.299162,58.85750,31.625000,5.000000,3.188750,1.650782e+09
std,190.386074,25.629693,111.300276,12.56525,7.405355,7.782765,1.990868,7.195820e+01
min,44.000000,-33.650000,-117.670900,42.62000,18.000000,0.000000,1.010000,1.650781e+09
25%,290.500000,30.806725,-107.425350,46.03250,29.000000,0.000000,1.875000,1.650781e+09
50%,343.000000,34.682400,46.765500,62.14500,34.000000,2.000000,2.485000,1.650782e+09
75%,392.250000,36.646050,110.396725,66.70500,37.250000,5.000000,4.057500,1.650782e+09
max,693.000000,50.364300,140.116700,74.88000,38.000000,22.000000,6.440000,1.650782e+09


## Hotel Map

In [48]:
# Extracting the city, country, latitude, and longitude from the ideal weather DataFrame
hotel_df = pd.DataFrame({'City': ideal_weather['City'], 'Country': ideal_weather['Country'],
                        'Lat': ideal_weather['Lat'], 'Lng': ideal_weather['Lng']})
                             
# Adding a blank column for hotel names
hotel_df['Hotel Name'] = ''

hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Busselton,AU,-33.6500,115.3333,
1,Krasnyy Chikoy,RU,50.3643,108.7512,
2,Ridgecrest,US,35.6225,-117.6709,
3,Praxédis Guerrero,MX,31.3667,-106.0167,
4,Flagstaff,US,35.1981,-111.6513,
5,Hūn,LY,29.1268,15.9477,
6,Akita,JP,39.7167,140.1167,
7,Leh,IN,34.1667,77.5833,


In [53]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Parameter dictionary for nearby search
params = {
    'location': '',
    'type': 'lodging',
    'radius': 5000,
    'key': g_key
}

# Using iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # Updating parameters with latitude and longitude for each city
    params['location'] = f'{row["Lat"]}, {row["Lng"]}'
    
    # Saving the request for each city as a json
    response = requests.get(base_url, params=params).json()
    
    # Assigning the name of the first hotel for each city in Hotel Name column
    try:
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
    except:
        # If there is none, then pass the city
        pass

In [54]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Busselton,AU,-33.6500,115.3333,Observatory Guest House
1,Krasnyy Chikoy,RU,50.3643,108.7512,Persona
2,Ridgecrest,US,35.6225,-117.6709,SpringHill Suites by Marriott Ridgecrest
3,Praxédis Guerrero,MX,31.3667,-106.0167,Congregación placitas de Otero
4,Flagstaff,US,35.1981,-111.6513,Drury Inn & Suites Flagstaff
5,Hūn,LY,29.1268,15.9477,Alrwasi Hotel
6,Akita,JP,39.7167,140.1167,Toyoko Inn Akita-eki Higashi-guchi
7,Leh,IN,34.1667,77.5833,Hotel Ladakh Greens


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))